In [ ]:
import networkx as nx
import pandas as pd
import numpy as np
import datetime
import random
import pickle
import os
import time
from decimal import Decimal as D
from decimal import ROUND_DOWN

import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.animation as animation

from lib.Line import Line
from lib.Station import Station
from lib.Platform import Platform
from lib.Passenger import Passenger
from lib.Train import Train
from lib.RailwayNetwork import RailwayNetwork

## Load raw data of passengers

Raw data with the number of in-out passengers per station and time window.

In [ ]:
DATA_PATH = '../Data/madrid/'
PARSED_PATH = DATA_PATH + 'parsed/'
IMAGES_PATH = DATA_PATH + 'images/'

if not os.path.exists(IMAGES_PATH):
    os.mkdir(IMAGES_PATH)

In [ ]:
connections = pd.read_csv(PARSED_PATH + 'station_connections.csv', index_col=False)
timetable = pd.read_csv(PARSED_PATH + 'timetable.csv', index_col=False)
w_lines = pd.read_csv(PARSED_PATH + 'line_weights.csv', index_col=False)
w_times = pd.read_csv(PARSED_PATH + 'times_weights.csv', index_col=False)
w_stations = pd.read_csv(PARSED_PATH + 'station_weights.csv', index_col=False)
station_coords = pd.read_csv(PARSED_PATH + 'station_coords.csv', index_col=False)

In [ ]:
# Transform strings to time objects
timetable['ARRIVAL_TIME'] = pd.to_datetime(timetable['ARRIVAL_TIME'])
timetable['ARRIVAL_TIME'] = [t.time() for t in timetable['ARRIVAL_TIME']]

timetable['DEPARTURE_TIME'] = pd.to_datetime(timetable['DEPARTURE_TIME'])
timetable['DEPARTURE_TIME'] = [t.time() for t in timetable['DEPARTURE_TIME']]

w_times['TIME'] = pd.to_datetime(w_times['TIME'])
w_times['TIME'] = [t.time() for t in w_times['TIME']]
w_times['WEIGHT_IN'] = [D(w) for w in w_times['WEIGHT_IN']]

w_lines['WEIGHT_IN'] = [D(w) for w in w_lines['WEIGHT_IN']]
w_lines['WEIGHT_OUT'] = [D(w) for w in w_lines['WEIGHT_OUT']]

w_stations['TIME'] = pd.to_datetime(w_stations['TIME'])
w_stations['TIME'] = [t.time() for t in w_stations['TIME']]
w_stations['WEIGHT_IN'] = [D(w) for w in w_stations['WEIGHT_IN']]
w_stations['WEIGHT_OUT'] = [D(w) for w in w_stations['WEIGHT_OUT']]

In [ ]:
# Cast values
connections = connections.astype({'DIRECTION': 'int32'})
timetable = timetable.astype({'IS_START': 'int32', 'IS_END': 'int32', 'DIRECTION': 'int32'})
station_coords = station_coords.astype({'X': 'float', 'Y': 'float'})

In [ ]:
# Set indexes
connections = connections.set_index(['LINE', 'FROM_STATION'])
timetable = timetable.set_index(['LINE', 'STATION_NAME'])
w_lines = w_lines.set_index(['LINE'])
w_times = w_times.set_index(['TIME'])
w_stations = w_stations.set_index(['LINE', 'STATION_NAME', 'TIME'])
station_coords = station_coords.set_index(['STATION_NAME'])

In [ ]:
w_stations.head()

## Build dictionaries

In [ ]:
station_names = connections.reset_index()['FROM_STATION'].unique()

In [ ]:
# Build dictionary: name of stations -> numerical ID
station2id = {st_name:i for i, st_name in enumerate(station_names)}
id2station = {i:st_name for i, st_name in enumerate(station_names)}

In [ ]:
# Example of getting a value of a line-station in a specific time
k_time = datetime.time(15, 0) # 15PM
w_stations.loc[('C5', 'ATOCHA', k_time), 'WEIGHT_IN']

## Build objects of network

In [ ]:
# Time of service (set to None if it never closes)
open_time = datetime.time(5, 0) # 5AM
close_time = datetime.time(1, 0) # 1AM

n_passengers = 400000

network = RailwayNetwork(n_passengers, open_time, close_time, w_times, w_lines, w_stations, timetable, connections)

In [ ]:
# Demo
s = Station(network, 'FUENLABRADA', w_stations)
print(s.get_weight(line_name='C5'))

In [ ]:
# Build station objects
stations = {st_name:Station(network, st_name, w_stations) for st_name in station_names}

In [ ]:
# Build line objects
lines = {line_name:Line(network, line_name, w_lines) for line_name in w_lines.index}

In [ ]:
# Once every object is created, add to them to the network and init it
network.init_network(lines, stations, debug=True)

In [ ]:
routes = timetable.reset_index(level=['LINE', 'STATION_NAME'])
routes = routes.loc[(routes['LINE'] == 'C5') &
                    (routes['STATION_NAME'] == 'FUENLABRADA') &
                    (routes['DIRECTION'] == 1)]
departure_times = routes.loc[routes['IS_END'] != 1]['DEPARTURE_TIME'].tolist()

departure_times.sort()

if departure_times[0] < network.close_time and departure_times[-1] > network.close_time:
    while departure_times[0] < network.close_time:
        t_departure = departure_times.pop(0)
        departure_times.append(t_departure)

departure_times

In [ ]:
"""with open(PARSED_PATH + 'network.pickle', 'wb') as fp:
    pickle.dump(network, fp, pickle.HIGHEST_PROTOCOL)"""

In [ ]:
"""with open(PARSED_PATH + 'network.pickle', 'rb') as fp:
    network = pickle.load(fp)"""

## Build graph

In [ ]:
G = nx.Graph()

G.add_nodes_from(station2id.values())

# Add attributes to nodes
for st_name, node_id in station2id.items():
    pos_x = station_coords.at[st_name, 'X']
    pos_y = station_coords.at[st_name, 'Y']
    G.add_node(node_id, pos=(pos_x, pos_y), station=stations[st_name])

# Add edges
for i, row in connections.iterrows():
    node_from = station2id[i[1]]
    node_to = station2id[row['TO_STATION']]
    
    G.add_edge(node_from, node_to)

In [ ]:
MAX_VALUE = 150

# Color mapping for nodes
cmap = plt.cm.jet  # define the colormap
# extract all colors from the .jet map
cmaplist = [cmap(i) for i in range(cmap.N)]
# force the first color entry to be grey
cmaplist[0] = (.5, .5, .5, 1.0)

# create the new map
cmap = mpl.colors.LinearSegmentedColormap.from_list('Custom cmap', cmaplist, cmap.N)
cmap.set_over('#811a17')

# define the bins and normalize
bounds = np.linspace(0, MAX_VALUE)
norm = mpl.colors.BoundaryNorm(bounds, cmap.N)

In [ ]:
fig = plt.figure(figsize=(12,12))
ax = fig.add_subplot(1,1,1)
ax.set_title('%sh' % network.get_time().strftime('%H:%M'))

pos = nx.get_node_attributes(G, 'pos')

colors = [s.get_n_passengers() for node_id, s in nx.get_node_attributes(G, 'station').items()]

ec = nx.draw_networkx_edges(G, pos, alpha=0.2)
nc = nx.draw_networkx_nodes(G, pos, nodelist=G.nodes, node_color=colors, 
                            with_labels=False, node_size=100, cmap=cmap, norm=norm, vmax=MAX_VALUE)
fig.colorbar(nc, extend='max')

## Run one-day simulation

In [ ]:
network.reset_network()

In [ ]:
# Display how the number of passengers change in a station

n_iters = 20 * 60 # 20 hours * 60 minutes

pos = nx.get_node_attributes(G, 'pos')

network.reset_network()

for i in range(n_iters):
    print('\r%d of %d' % (i + 1, n_iters), ' '*10, end='')
    
    network.step()
    
    fig = plt.figure(figsize=(12,12))
    ax = fig.add_subplot(1,1,1)
    
    ax.set_title('%sh' % network.get_time().strftime('%H:%M'))

    colors = [s.get_n_passengers() for node_id, s in nx.get_node_attributes(G, 'station').items()]

    ec = nx.draw_networkx_edges(G, pos, alpha=0.2)
    nc = nx.draw_networkx_nodes(G, pos, nodelist=G.nodes, node_color=colors, 
                                with_labels=False, node_size=100, cmap=cmap, norm=norm, vmax=MAX_VALUE)
    fig.colorbar(nc, extend='max', ax=ax)
    
    fig.savefig('%s%s.png' % (IMAGES_PATH, network.get_time().strftime('%H-%M')))
    plt.close(fig)
    
print('Finish!', ' '*10)